# AI Agent

In [ ]:
!pip install crewai==0.30.0 crewai_tools==0.2.3 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
# from utils import get_openai_api_key, get_serper_api_key

openai_api_key = 'Yourkey'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = 'Yourkey'
os.environ["OPENAI_API_KEY"] = openai_api_key

## crewAI Tools

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, YoutubeVideoSearchTool,PDFSearchTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
youtube_tool = YoutubeVideoSearchTool()
pdf_tool = PDFSearchTool()


## Creating Agents

In [ ]:
# import os
# os.kill(os.getpid(), 9)

In [ ]:
data_analyst_agent = Agent(
    role="United Airlines Reservation Agent",
    goal="Refund a customer ({question}) according to the ({url})",
    backstory="Specializing in refunding, this agent "
              "uses the ({url}) United which can be searched online"
              "to provide crucial insights on refunding for customers",
    verbose=True,
    allow_delegation=False,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [ ]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Read and understand the United Refund Page. When user ask ({question}), provide them answer promptly according to provided url ({url})"
    ),
    expected_output=(
        "answer from user({question}) "
    ),
    agent=data_analyst_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [ ]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
refund_crew = Crew(
    agents=[data_analyst_agent],

    tasks=[data_analysis_task],

    manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
# Example data for kicking off the process
refund_crew_inputs = {
'question': 'I cancelled my flight 2 hours prior to departure time, can I still get a refund?',
'url' :'https://www.united.com/en/us/'
}

In [ ]:
### this execution will take some time to run
result = refund_crew.kickoff(inputs=refund_crew_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Read and understand the United Refund Page. When user ask (I cancelled my flight 2 hours prior to departure time, can I still get a refund?), provide them answer promptly according to provided url (https://www.united.com/en/us/)


> Entering new CrewAgentExecutor chain...
I need to gather all necessary information from the United Refund Page in order to provide a prompt and accurate answer to the user's question.

Action:
Ask question to co-worker

Action Input:
{
"coworker": "United Airlines Reservation Agent",
"question": "Can a passenger receive a refund if they cancel their flight 2 hours prior to departure time?",
"context": "I need to know the refund policy for United Airlines when a passenger cancels their flight 2 hours prior to departure time."
}


> Entering new CrewAgentExecutor chain...
I should check the United Airlines website to find the specific refund policy for passengers who cancel their flight 2 hours pri

- Display the final result as Markdown.

In [ ]:
from IPython.display import Markdown
Markdown(result)

United Airlines' refund policy states that passengers may be eligible for a refund if they cancel their flight within 24 hours of booking and at least 7 days before the departure date. Unfortunately, canceling a flight 2 hours prior to departure time may not qualify for a refund according to United's policy.